# Импорт нужных библиотек

In [1]:
# избавляемся от полотна ошибок
import warnings
warnings.filterwarnings("ignore")

# основные библиотеки для построения моделей
import pandas as pd
import numpy as np
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchvision import transforms as T

# библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from tqdm import tqdm
from PIL import Image

# вспомогательные библиотеки
import os
import shutil
from random import shuffle

sns.set_style('whitegrid')
plt.rcParams.update({'font.size': 15})

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [52]:
device

'cuda'

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Объявление необходимых классов работы с моделью

## Класс модели

In [5]:
class MOLI_Net(nn.Module):
    def __init__(self, img_channels=3, num_classes=6):
        super().__init__()
        # обычно эту часть называют «тушкой модели»
        self.encoder = nn.Sequential(
            # 1
            nn.Conv2d(in_channels=img_channels, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(64),
            # экскурс: stride - шаг, с которым прикладываем ядро свёртки
            # паддинг - размер доп обрамления (чтобы учитывать края изображения при свёртке)
            # функция активации
            nn.ReLU(),
            # ещё одна техника, фактически проходимя матричкой 2x2 и берём максимум, делаем с шагом stride
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            # 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            # 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            # 4
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            # 5
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            # 6
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            # 7
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            # 8
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        )
        # обычно называют «головой», тк это полносвязные сети, которые могут уловить больше необходимой информации
        self.head = nn.Sequential(
            # полносвязный слой
            nn.Linear(in_features=512, out_features=4096),
            nn.ReLU(),
            # dropout механизм «выкидывания» нейронов из сетки с вероятностью p
            nn.Dropout(p=0.5),

            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=4096, out_features=num_classes)
        )

    def forward(self, x):
        out = self.encoder(x)
        # растягиваем матричку в вектор
        out = nn.Flatten()(out)
        out = self.head(out)
        return out

## Класс даталоадера

In [135]:
for_resize = (32, 32)
base_transforms = [T.Resize(for_resize), T.ToTensor()]
# пути под данные
try_path = '/content/drive/MyDrive/try'

# метки для изображений
labels = {
    'binom': 0,
    'geom': 1,
    'poisson': 2,
    'norm': 3,
    'pareto': 4,
    'vigner': 5
}

mir_labels = {
    0: 'binom',
    1: 'geom',
    2: 'poisson',
    3: 'norm',
    4: 'pareto',
    5: 'vigner'
}

In [138]:
# класс собственного dataset
class dataset_statistic(torch.utils.data.Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.files = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.files[index])
        image = Image.open(img_path)
        label = labels[img_path.split('/')[-1].split('_')[0]]
        if self.transforms:
            for t in self.transforms:
                image = t(image)
        return (image, torch.tensor(label))

In [123]:
# функция для отрисовки изображения
def show_image(data):
    img = data[0]
    label = data[1].item()
    print(mir_labels[label])
    plt.grid('off')
    plt.axis('off')
    plt.imshow(img.permute(1, 2, 0))

# Скачивание лучшей модели

In [35]:
model = MOLI_Net()
model.load_state_dict(torch.load('/content/drive/MyDrive/color_model.h5'))
model.to(device)

MOLI_Net(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Формирование набора, для которого хотим предсказывать

In [141]:
try_data = dataset_statistic(try_path, transforms=base_transforms)
try_data_loader = torch.utils.data.DataLoader(try_data, batch_size=32, shuffle=False)

# Сценарий работы


In [88]:
model.eval()

MOLI_Net(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [154]:
for images, targets in try_data_loader:
    # скидываем на gpu
    images = images.to(device)
    logits = model(images)
    target_pred = torch.max(logits, dim=1)[1].cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()

In [155]:
true_answers = [mir_labels[x] for x in targets]
pred_answers = [mir_labels[x] for x in target_pred]

# Функция для получения ответа

In [161]:
def give_answers(path):
    for_resize = (32, 32)
    base_transforms = [T.Resize(for_resize), T.ToTensor()]
    data = dataset_statistic(path, transforms=base_transforms)
    data_loader = torch.utils.data.DataLoader(try_data, shuffle=False)

    model.eval()
    for images, targets in try_data_loader:
        images = images.to(device)
        logits = model(images)
        target_pred = torch.max(logits, dim=1)[1].cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()

    true_answers = [mir_labels[x] for x in targets]
    pred_answers = [mir_labels[x] for x in target_pred]

    # print(true_answers)
    # print(pred_answers)

    return pred_answers

In [162]:
give_answers('/content/drive/MyDrive/try')

['vigner', 'geom']